
# Part 1: 基于多层感知机的从零开始手写数字识别

在实验part1中, 我们希望从头开始做一个手写数字识别的多层感知机: 只使用 `NumPy` 和 `matplotlib` 两个库.

**Note:** 每个模块都可以验证结果

## Import
首先，导入两个库。`Numpy`用来进行矩阵运算，`matplotlib`用来进行画图。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 数据准备 
我们的数据来自 **MNIST** 数据集，里面是28x28像素从0-9的手写数字图片。数据位于`data`文件夹内压缩包中，需要自行下载解压。

### 加载数据
解压后的**data**文件夹中已经有两个csv文件: **mnist_train.csv**和**mnist_test.csv**分别用来训练和测试。首先，我们需要加载数据。我们新建一个名`LoadData`的函数来加载数据。 

我们在训练集中有60000个样本和10000个测试样本。每一个28x28的图片都被解压为1维向量，28 x 28 = 784。 `LoadData()` 函数加载数据进两个array中 $X$ 和 $Y$。我们可以通过参数`limit`来控制用来测试和训练的输入例子的数量，上限是csv文件最大样本数。

In [ ]:
def LoadData(DirName,limit = 500):
    data = list();
    count = 0;
    with open(DirName) as f:
        for line in f:
            if count<limit:
                new_line = np.array(line.split(','))
                new_line = new_line.astype(np.float32)
                data.append(new_line)
                count += 1
    data = np.asarray(data)
    
    return data[:,0],data[:,1:]

NoTrain = 60000
NoTest = 10000
Y_train,x_train = LoadData("../data/mnist_train.csv", NoTrain)
Y_test,x_test = LoadData("../data/mnist_test.csv", NoTest)

print(f"Test set size: {x_test.shape[0]} x {x_test.shape[1]}")
print(f"Train set size: {x_train.shape[0]} x {x_train.shape[1]}")

### MNIST 可视化
下面的代码只是想展示数据集的样子，变量`sample`定义其中一个例子。这里也把平面784个数据数据重新变成28x28的图片矩阵并与`matplotlib`库中的`imshow()`函数兼容。

In [ ]:
sample = 7;  # index of the image we want to show

plt.title(f'The handwritten digit is {Y_train[sample]:.0f}')

# Reshape the array into 28 x 28 array (2-dimensional array)
pixels = x_train[sample,:].reshape((28, 28))

plt.imshow(pixels, cmap='gray',vmin=0,vmax=1.0)
plt.show()

### 重改尺寸和编码数据
#### 重改输入数据特征值
输入数据($X$ array)的范围从 0 到 255 (范围很大)。因此，为了减少这么大的范围，我们需要重新改变数据的尺寸。一个常规的方式是标准化方法 即重改数据的平均值为零，标准差为一。
下面是标准化的公式:
$$
X^{\prime}=\frac{X-\mu}{\sigma},
$$
其中 $\mu$ 是特征值的平均值，$\sigma$是训练特征值的标准差。

In [ ]:
# standardize training and test data
x_train_st = (x_train-np.mean(x_train))/np.std(x_train)
x_test_st = (x_test-np.mean(x_train))/np.std(x_train)

print(f"Your training data mean is {np.mean(x_train_st):.3f} and your standard deviation is {np.std(x_train_st):.3f}")
print(f"Correct training data mean is {-0.000:.3f} and correct standard deviation is {1.000:.3f}")
print(f"Your test data mean is {np.mean(x_test_st):.3f} and your standard deviation is {np.std(x_test_st):.3f}")
print(f"Correct test data mean is {0.006:.3f} and correct standard deviation is {1.008:.3f}")

#### 输出编码
此外，对于分类数据(其中图像被标记为0-9)，我们需要数据的独热编码表示，这意味着每个标签被转换为一个二进制向量（例如，标签9对应的向量是[0,0,0,0,0,0,0,0,0,1]）。

In [ ]:
# One-hot encoding train and test sets labels 
y_train = np.zeros((Y_train.size, int(Y_train.max()) + 1))
y_train[np.arange(Y_train.size),np.array(Y_train,dtype='int')] = 1.0;

y_test  = np.zeros((Y_test.size, int(Y_test.max()) + 1))
y_test[np.arange(Y_test.size),np.array(Y_test,dtype='int')] = 1.0;

print(f"Your decimal label is {Y_train[0]:.0f} and your one-hot encoded label is {y_train[0,:]}")
print(f"Correct decimal label is 5 and correct one-hot encoded label is [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]")

## 神经网络(Neural network,NN)
神经网络(NN)是由称为人工神经元的节点组成的连接层的集合。每个连接都向其他神经元传递一个“信号”。一种人工神经元，它接收信号，然后对信号进行处理，并可以向与之相连的神经元发出信号 [[1]](https://www.kaggle.com/code/androbomb/simple-nn-with-python-multi-layer-perceptron).
连接处的“信号”是一个实数，每个神经元的输出由它的输入之和的激活函数来计算。信号从第一层(输入层)传输到最后一层(输出层)。
位于输入层和输出层之间的隐藏层中的神经元从输入层中的神经元或者从更早的隐藏层中的神经元接收输入。

## 多层感知机Multi-layer perceptrons (MLP)
最著名的NN模型是多层感知器（MLP），它是一个前馈全连接的网络。

MLP模型有完全连接的层，这样每个输入神经元都与下一层的每个神经元相连，信息只向前移动（没有反馈）。因此，神经元将一个层的输入值移动到下一个层。在数学上，MLP模型定义了一个参数化的函数，其中每一层都计算一个加权线性组合, $h =XW+b$ 的层输入由向量 $X$表示, 然后是一个激活函数 $\sigma(h)$. MLP配置可以重复连锁，建立一个深度神经网络（DNN）。

下图显示了具有一个隐藏层的简单MLP的结构。

<img src="mlp.jpg" width="600">

Image credit: [https://github.com/PetarV-/TikZ/tree/master/Multilayer%20perceptron](https://github.com/PetarV-/TikZ/tree/master/Multilayer%20perceptron)

$W_i$表示连接从第 $(i − 1)$层到第 $i$层的权重也决定了这两层之间联系的强度和符号。除了权重之外，通常还有一个偏置项（用$b$表示）来调整偏移。
<!---For the sake of simplicity, here, we add an additional constant $x_0 =1$ concatenated to each input to make the $w_0 \sim b$.
(__Note:__, Therefore, in our application to MNIST each input would be 784+1 dimensional (including the bias))-->
第$i$层的输出用 $O^i$表示, 公式如下:
$$
O^{(i)} = \sigma\left(O^{(i-1)}W^{(i)}+b^{(i)}\right),
$$
其中 $\sigma(x)$ 是对应层的激活函数。输入层的输出是$O^{(0)}$ ，输入是向量$X$。在我们的例子中，输出层有10个维度，用于表示输出标签的一热编码二进制向量。

### 激活函数
激活函数 $\sigma(\cdot)$ 是一个非线性函数。有多种激活函数可用于不同的NN模型。 在本实验中，我们对隐藏层和输出层分别使用两个激活函数**sigmoid** 和 **ReLU**。

__sigmoid:__$$\sigma(x)=\frac{1}{1+e^{-x}}$$
__ReLU:__ $$ \sigma(x)= \begin{cases}0, &  x<0, \\ x, &  x\geq 0.\end{cases}
$$

### 前向传递

从输入产生输出的过程被称为前向传递。对于MLP来说，前向传递基本上是一系列的矩阵乘法。

现在，我们想为MNIST数据集的数字识别问题定义一个具有一个隐藏层的MLP。下面是所需的MLP模型的图：

<img src="nn_mnist.jpg" width="600">

__注:__ 图中的上标$i, i={0,1,2}$ 表示相关的层号。此外，为了简单起见，图中只规定了每层中第一个神经元的所需计算和输出。每层中的其余神经元的结构与第一个神经元相同。

隐藏层有128个神经元。如前所述，隐藏层的激活函数与输出层不同。 

如图所示，隐藏层中每个神经元的输出将被计算如下：
$$
O^{(1)}_{j}=\text{sigmoid}\left(\sum_{i=0}^{783} x_i \times w^{(0)}_{i, j}\right) \; \text{ for } \; j={1,2,...,128}
$$
(__注:__ 在这个例子中，没有偏置项). 
如图所示，权重$w^{(0)}_{i,j}$需要被压缩成矩阵 $W^{(0)}\in \mathbb{R}^{784\times128}$来允许隐蔽层的所有神经元使用矩阵乘法进行有效计算。  由于需要一个不同的权重矩阵来转换隐藏层到输出层, $W^{(0)}$ 是转换输入到隐藏层的矩阵:
$$
O^{(1)} = \text{sigmoid}\left(h^{(0)}\right), \text{ where } h^{(0)}=XW^{(0)}  
$$
同样, $W^{(1)}$ 将隐藏层转换为输出层，如下所示:
$$
O^{(2)}_{j}=\text{ReLU}\left(O^{(1)}W^{(1)}\right), O^{(1)} \in \mathbb{R}^{1\times 128}, W^{(1)} \in \mathbb{R}^{128\times 10} 
.$$

分类任务的最后一步是从输出层的$O^{(2)}_j, j=\{0,1,...,M-1\}$产生一个预测, 其中 $M$ 是标签数量(在MNIST数据集中$M=10$). 通常，对于多标签分类，**softmax** 函数被用来将输出层中$M$神经元的输出转换为概率，如下所示:

$$
\text{softmax}(z_j) =\frac{e^{z_j}}{\sum^{M-1}_{i=0}e^{z_i}},
$$

这个函数的有趣属性是，softmax的所有输出之和总是等于1:

$$
\sum^{M-1}_{i=0}\text{softmax}({z_i}) = 1
$$

因此，对我们例子的输出层应用softmax，可以有效地将神经元的输出转换为向量$hat{y}$中每个标签的概率，如下所示:

$$
\hat{y}_j = \text{softmax}\left({O^{(2)}_j}\right), \; j\in \{0,1,...,M-1\}.
$$

现在，我们已经重新审视了MLP模型的前向传递背后的数学知识。那么，让我们开始在python代码中实现我们所看到的一切吧！

### MLP 实现

#### 输入初始化
由于我们希望写出高效快速的代码，我们尽可能使用矩阵（NumPy多维数组）操作。在对NumPy数组进行许多操作时，NumPy会自动采用并行性，这将有助于大幅加速我们的代码（这有时也被称为代码的矢量化）。

输入向量$X$, 权重矩阵 $W^{(0)}$ 和 $W^{(1)}$ 和激活函数是我们定义前向传递所需要的。

In [ ]:
X = x_train_st 
Y = y_train 

N_l = 128  # number of neurons in hidden layer

layers = np.array([X.shape[1]]+[N_l]+[Y.shape[1]]) # layers = [784 128 10]

print('Your', '\033[1m' + 'layers' + '\033[0m','array is:', layers)
print('Correct ', '\033[1m' + 'layers' + '\033[0m','array is: [784 128  10]')

#### 权重初始化
为了进行前向传递，我们的输入向量$X$连续乘以权重矩阵，然后传递给相应的激活函数。

权重矩阵的值最终将通过反向传播学习，但每个权重矩阵必须首先被初始化为随机值。

有几种不同的方法来进行这种初始化。在这里，我们使用一个简单的方法，从[-1,1]范围上的均匀分布开始取值。

In [ ]:
# Weights for feedforward
np.random.seed(seed=0)

def init_weights(layers):
    weights = list()
    # for hidden layer
    weights_l1 = np.random.uniform(-1,1,(784,128))
    weights.append(weights_l1)
    #for output layer
    weights_l2 = np.random.uniform(-1,1,(128,10))
    weights.append(weights_l2)
    
    weights = np.asarray(weights,dtype=object)
    return weights

weights = init_weights(layers)

print(f"Dimension of your weight matrix for the hidden layer is {weights[0].shape[0]} x {weights[0].shape[1]} and for the output layer is {weights[1].shape[0]} x {weights[1].shape[1]}")
print("Dimension of the correct weight matrix for the hidden layer is 784 x 128 and for the output layer is 128 x 10")

print(f"Your initialized weight connecting the first neuron in the hidden layer to the first neuron in the output layer is {(weights[1][0][0]):.6f}")
print("Correct initialized weight connecting the first neuron in the hidden layer to the first neuron in the output layer is 0.705750")


#### 激活函数

这里，我们将定义我们的网络所需的激活函数，即 `ReLU()`, `sigmoid()`, and `softmax()` 三种函数.

In [ ]:
def ReLU(x):
    """
    Input Parameters:
    x : input: float of array
    
    Returns:
    ReLU(x) : float or array
    """
    y = np.maximum(0,x)
    return y

def sigmoid(x):
    """
    Input Parameters:
    x : input: float of array
    
    Returns:
    sigmoid(x) : float or array
    """
    y = 1/(1+np.exp(-x))
    return y

def softmax(x):  
    """
    Input Parameters:
    x : input: array(n x p) : n samples by p dimensions : p=10 for MNIST (because we have 0-9 digits)
    
    Returns:
    softmax(x) : float or array
    """
    y = np.exp(x)/np.sum(np.exp(x),axis=1, keepdims=True)
    return y

print(f"Your outputs: ReLU(-0.1) = {ReLU(-0.1):.3f}, ReLU(0.1) = {ReLU(0.1):.3f}, sigmoid(-0.1) = {sigmoid(-0.1):.3f}, sigmoid(0.1) = {sigmoid(0.1):.3f}, softmax([0.1 0.2] =", softmax([[0.1, 0.2]]))
print("Correct outputs: ReLU(-0.1) = 0.000, ReLU(0.1) = 0.100, sigmoid(-0.1) = 0.475, sigmoid(0.1) = 0.525, softmax([0.1 0.2] = [[0.47502081 0.52497919]]) ")

### 层初始化
隐蔽层和输出层的值将被存储在多维矩阵中。

In [ ]:
# Initialization of the layer outputs
n_samples = X.shape[0]

def init_layer(layers,n_samples):
    h = [np.empty((n_samples,layers[1])),np.empty((n_samples,layers[2]))] 
    o = [np.empty((n_samples,layers[0])),np.empty((n_samples,layers[1])),np.empty((n_samples,layers[2]))] 
    return h,o

h,o = init_layer(layers,n_samples)

### 前向传递
为了简单起见，我们要求你实现特定MLP的硬编码版本，也就是说，你的代码不需要对任何层数、神经元等进行通用。因此，你只需要实现矩阵乘法并为每一层应用所要求的激活函数。

In [ ]:
def feedforward(data,weights,o,h):
    
    # Input layer
    o[0] = data
    
    # Hidden layer --> ReLU activation
    h[0] = np.matmul(o[0],weights[0])
    
    o[1] = ReLU(h[0])
    
    # Output Layer --> sigmoid + softmax activations
    h[1] = np.matmul(o[1],weights[1])
    o[2] = sigmoid(h[1])
    Y_hat = softmax(o[2])

    return Y_hat

Y_hat = feedforward(X,weights,o,h)

with np.printoptions(precision=4, suppress=True):
    print('Your predicted output for the first training sample is:',Y_hat[0])
print('Correct predicted output for the first training sample is: [0.0593 0.1611 0.0593 0.1611 0.0593 0.0593 0.1611 0.0593 0.1611 0.0593]')


## 损失函数
损失函数量化了神经网络的输出$\hat{\mathbf{y}}$与预期输出$\mathbf{y}$相比的 "错误 "程度，被用来评估神经网络的性能。
### Cross-entropy loss function
损失函数应该反映定义的神经网络的目标。例如，在我们的设计中，我们希望对0-9的手写数字进行分类。交叉熵损失，也被称为负对数可能性，最常用于分类问题。下面是这个函数的数学表示法:
$$
\text{L}(\mathbf{y},\hat{\mathbf{y}}) = -\sum_{k=0}^{M-1}{y_k \log(\hat{y}_k)},
$$
其中 $M$是不同标签的数量下面的方程被用来定义$N$输入样本的成本函数:
$$
\text{C} = \sum_{j=1}^{N}{\text{L}(\mathbf{y}^{(i)},\hat{\mathbf{y}}^{(i)})}.
$$
下面的函数计算了总体成本函数 $C$。

In [ ]:
#cross entropy loss  
def cross_entropy(y_pred,y):
    """
    Input Parameters:
    y_pred, y : array of float
    
    Returns:
    c : float
    """
    
    # Compute loss
    c = sum(-1*np.sum(y*np.log(y_pred),axis = 1))
    return c
loss = cross_entropy(Y_hat,Y)
print(f"Your loss with randomly initilized weights is {loss:.3f}")
print(f"Correct loss with randomly initilized weights is 143488.412")

### 准确率

输入通常是根据可能性最大的输出标签来分类。

$$
\text{prediction}(x) = \text{argmax}(\hat{\mathbf{y}}) 
$$

因此，为了评估我们模型的分类准确性，我们必须确定概率最高的输出标签，给该标签赋予 "1"，并将其余标签设置为 "0"。然后，我们可以通过比较数据集中的预测标签和实际标签来计算我们分类器的准确性。下面的两个函数实现了这个功能。

In [ ]:
def labeling(x):  
    # Set the label with the max probability to '1' and the rest to 0 
    label = np.zeros((x.shape[0],Y.shape[1]))
    label[np.arange(x.shape[0]),x.argmax(axis=1)] = 1
    return label

def accuracy(y_pred,y):  
    # Calculate the accuracy along the rows, averaging the results over the number of samples.
    acc = np.mean(np.equal(y_pred.argmax(axis=1),y.argmax(axis=1)).astype(int))
    
    return acc

acc = accuracy(labeling(Y_hat),Y)
print(f"Your accuracy with randomly initialized weights is {acc:.4f}")
print(f"Correct accuracy with randomly initialized weights is 0.1163")

## 训练

迭代更新网络的权重以提高性能的过程被称为训练。

### 梯度下降
如前所述，神经网络的性能是用损失函数$C$来评估的。

因此，我们的目标是最小化与神经网络权重有关的损失函数。这可以用 __梯度下降(gradient descent,GD)__ 算法来解决，定义更新规则如下:
$$
W^{(l)}_t = W^{(l)}_{t-1}-\eta\frac{\partial C}{\partial W^{(l)}_{t-1}},
$$
其中 $t$ 是迭代次数, $\eta$ 是用于调整GD算法步骤的学习率, and $\frac{\partial C}{\partial W^{(l)}_{t}}$ 是损失函数相对于第$(l-1)$层的神经元和第$l$层的神经元连接的权重矩阵的偏导。该算法朝着成本函数$C$的最小值迈进。 每一步的大小与梯度的大小成正比，梯度是成本函数相对于每个参数的偏导。 

由于和的导数等于导数之和，我们可以将更新规则改写如下:
$$
W^{(l)}_{t} = W^{(l)}_{t-1}-\eta\sum_{i=1}^{N}\frac{\partial L_i}{\partial W^{(l)}_{t-1}},
$$
其中 $L_i=L\left(\mathbf{y}^{(i)},\hat{\mathbf{y}}^{(i)}\right)$ 是训练集中第$i$层样本的的交叉熵值损失。一旦我们计算出对于每个训练样本的偏导数 $\frac{\partial L_i}{\partial W^{(l)}_{t-1}}$, 我们可以将所有单独的导数项相加，计算出 $\frac{\partial C}{\partial W^{(l)}_{t-1}}$.

### 反向传播
为了对我们的例子进行GD，我们需要为输出层和隐藏层的权重矩阵分别计算梯度 $\frac{\partial C}{\partial W^{(1)}}$ 和 $\frac{\partial C}{\partial W^{(0)}}$(注意我们为了简化记号放弃了下标$t$ 和 $i$)。反向传播是一种有效计算这些梯度的算法。

为了计算 $\frac{\partial L}{\partial W^{(1)} }$, 我们使用链式法则:
$$
\frac{\partial L}{\partial W^{(1)}} = \frac{\partial L}{\partial h^{(1)}}\times \frac{\partial h^{(1)}}{\partial W^{(1)}},
$$
其中 $\frac{\partial h^{(1)}}{\partial W^{(1)}} = O^{(1)}$ 因为 $h^{(1)}=O^{(1)}W^{(1)}$.

对于 $\frac{\partial L}{\partial h^{(1)}}$, 让我们再次应用连锁规则，定义之后会用到的 $\delta_1$:
$$
\delta_1 \triangleq \frac{\partial L}{\partial h^{(1)}} =  \frac{\partial L}{\partial O^{(2)}} \times  \frac{\partial O^{(2)}}{\partial h^{(1)}}.
$$
其中$\frac{\partial L}{\partial O^{(2)}}$ 是交叉熵损失函数相对于softmax函数的输入的部分导数,并且它等于 $\hat{y}-y$ (见详细解释 [here](https://www.mldawn.com/back-propagation-with-cross-entropy-and-softmax/))。因为 $O^{(2)} = \text{sigmoid}\left(h^{(1)}\right)$, 我们有 $\frac{\partial O^{(2)}}{\partial h^{(1)}} = \text{sigmoid}\left(h^{(1)}\right)\left(1-\text{sigmoid}\left(h^{(1)}\right)\right)$, 它是sigmoid激活函数相对于其输入的导数。 (见详细解释 [here](https://scribe.esmailelbob.xyz/derivative-of-the-sigmoid-function-536880cf918e)). 因此，把所有东西放在一起，我们有:
$$
\frac{\partial L}{\partial W^{(1)}} = {\left(O^{(1)} \right)}^{T} \left(\hat{y}-y\right)\text{sigmoid}\left(h^{(1)}\right)\left(1-\text{sigmoid}\left(h^{(1)}\right)\right)
$$
__注:__ 对于相对于矩阵的导数，我们需要对矩阵的维度加以注意。更详细资料见 [this video](https://www.youtube.com/watch?v=GlcnxUlrtek&list=PLiaHhY2iBX9hdHaRr6b7XevZtgZRa1PoU&index=5).

之后, 我们需要为隐藏层以同样的方式计算 $\frac{\partial L}{\partial W^{(0)} }$ 。再次,我们应用链式法则:
$$
\frac{\partial L}{\partial W^{(0)}} = \frac{\partial L}{\partial h^{(0)}}\times \frac{\partial h^{(0)}}{\partial W^{(0)}}= {\left(O^{(0)}\right)}^{T} \times \delta_0\\
\delta_0 \triangleq \frac{\partial L}{\partial O^{(1)}} \times \frac{\partial O^{(1)}}{\partial h^{(0)}} = \frac{\partial L}{\partial O^{(1)}} \times \text{ReLU}'(h^{(0)}),
$$
其中 $\text{ReLU}'(h^{(0)}) = \begin{cases} 0, & h^{(0)} \leq 0, \\ 1, & h^{(0)} > 1,\end{cases}$ 是ReLU激活函数相对于数入的导数。 继续使用链式法则和之前定义的$\delta_2$, 我们有:
$$
\frac{\partial L}{\partial O^{(1)}} = \frac{\partial L}{\partial h^{(1)}} \times \frac{\partial h^{(1)}}{\partial O^{(1)}} = \delta_1 \times \left({W^{(1)}}\right)^{T}.
$$
所以最后我们有:
$$
\frac{\partial L}{\partial W^{(0)}} = {\left(O^{(0)}\right)}^{T} \times \delta_1 \times {\left(W^{(1)}\right)}^{T} \times \text{ReLU}'(h^{(0)}) 
$$

### 反向传播实现
现在，我们要对数据集中的一个样本实现反向传播。首先，我们实现 **sigmoid** 和 **ReLU** 激活函数:

In [ ]:
def sigmoid_prime(x):
    # Compute the derivative of sigmoid where h=sigmoid(x)
    derivative = sigmoid(x)*(1-sigmoid(x))
    return derivative

def ReLU_prime(x):
    # Derivative of ReLU function
    derivative = np.where(x > 0, 1, 0)
    return derivative

print(f"Your sigmoid derivative at x = 0.1 is {sigmoid_prime(0.1):.3f}")
print(f"Correct sigmoid derivative at x = 0.1 is 0.249")
print(f"Your ReLU derivative at x = 0.1 is {ReLU_prime(0.1):.3f} and at at x = -0.1 is {ReLU_prime(-0.1):.3f}")
print(f"Correct ReLU derivative at x = 0.1 is 1.000 and at at x = -0.1 is 0.000")

然后, 我们对输出层计算 $\delta_1$ and $\frac{\partial L}{\partial W^{(1)}}$ 和对隐藏层计算 $\delta_0$ and $\frac{\partial L}{\partial W^{(0)}}$, 暂时只针对我们训练集的第一个样本。

In [ ]:
# Calculate d(L,w1)
delta_1 = (Y_hat[0]-Y[0])*sigmoid_prime(h[1][0])
d_l_w1 = np.matmul(o[1][0].reshape(128,-1),delta_1.T.reshape(-1,10))

# Calculate d(L,w0)
delta_0 =np.matmul(delta_1,weights[1][0].T)*ReLU_prime(h[0][0])
d_l_w0 =np.matmul(o[0][0].reshape(784,-1),delta_0.T.reshape(-1,128))

print(f"Your derivative of the cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer is: {(d_l_w1[0][0]):.10f}")
print('Correct derivative of the cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer is: 0.0006078749')
print(f"Your derivative of the cost function with respect to W^(0) connecting the first neuron of the input layer to the first neuron of the hidden layer is: {(d_l_w0[0][0]):.10f}")
print('Correct derivative of the cost function with respect to W^(0) connecting the first neuron of the input layer to the first neuron of the hidden layer is: -0.0000148791')

正如之前所说, 为了计算 $\frac{\partial C}{\partial W^{(l)}}$, 我们需要添加所有的单个导数 $\frac{\partial L_i}{\partial W^{(l)}}$ ，$i = \{1,\dots,N\}$。 我们可以很容易地用矩阵乘法做到这一点，通过用 $N \times M$ 矩阵 $Y$ 和 $\hat{Y}$来代替$1 \times M$向量 $y$ 和 $\hat{y}$ 。

在这里，我们重新使用前一个单元的梯度计算来实现完整的 `backpropagation()` 函数, 包括计算 $\frac{\partial C}{\partial W^{(1)}}$ 和 $\frac{\partial C}{\partial W^{(0)}}.$

In [ ]:
def backpropagation(Y,Y_hat,weights,h,o):
    
    delta = [np.empty((Y.shape[0],weights[0].shape[1])),np.empty((Y.shape[0],weights[1].shape[1]))]
    dC_dw = [np.empty((weights[0].shape[0],weights[0].shape[1])),np.empty((weights[1].shape[0],weights[1].shape[1]))]
    
    # Calculate d(L,w1)
    delta[1] = (Y_hat-Y)*sigmoid_prime(h[1])
    dC_dw[1] = np.matmul(o[1].T,delta[1])
    
    # Calculate d(L,w0)
    delta[0] = np.matmul(delta[1],weights[1].T)*ReLU_prime(h[0])
    dC_dw[0] = np.matmul(o[0].T,delta[0])
    
    return dC_dw

# Test backpropagation for first sample
dl_dw_index = backpropagation(Y[0].reshape(-1,10),Y_hat[0],weights,h,o)

print(f"Your derivative of the cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer for the first sample is: {(dl_dw_index[0][0][0]):.3f}")
print('Correct derivative of the cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer for the first sample is: -23.102')

# Test backpropagation for the entire training dataset
dl_dw_index = backpropagation(Y,Y_hat,weights,h,o)

print(f"Your derivative of cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer for the entire dataset is: {(dl_dw_index[0][0][0]):.3f}")
print('Correct derivative of cost function with respect to W^(1) connecting the first neuron of the input layer to the first neuron of the hidden layer for the entire dataset is: -3.043')

## Mini-batch 梯度下降
批量梯度下降（Batch gradient descent，GD）需要在更新权重之前对所有的训练集$N$样本计算梯度 $\frac{\partial L_i}{\partial W^{(l)}}$。 我们采用了一种被称为Min-bathc GD的改进版，它使用训练集 $B$ 样本的非重叠子集 $\mathbb{B}$ 而不是所有的 $N$ 样本 来计算梯度和更新权重:
$$
W^{(l)}_{k} = W^{(l)}_{k-1}-\eta\sum_{i\in \mathbb{B}}\frac{\partial L_i}{\partial W^{(l)}_{k-1}}
$$
在Mini-batch GD的每一步$k$，我们使用具有当前内部参数集（在我们的例子中为权重）的模型对$B$样本进行预测，将预测结果与实际预期结果进行比较，计算损失，并使用损失通过反向传播更新内部模型参数。一旦所有的小批处理完毕，一个所谓的训练 *周期（epoch）* 就已经过去了，我们将训练集吐到一个新的小批中，继续训练。

从实现的角度来看，训练过程是一个嵌套的for循环，其中外循环迭代训练历时，内循环迭代小批并更新权重。请注意，通常在每个历时结束时随机洗刷训练样本，以便在每个历时中创建不同的小批次。此外，我们要存储和绘制训练和测试损失（分别为训练和测试样本的平均值），以及训练和测试精度作为历时数的函数。这些图可以帮助诊断模型是过度拟合还是不足，以及各种参数选择是否有意义。

In [ ]:
## Mini-batch gradient descent
np.random.seed(seed=0)

X_test = x_test_st 
Y_test = y_test
n_test = X_test.shape[0]

# lists for saving the value of loss and accuracy of each epoch
mean_train_loss_list = list()
train_acc_list = list()
mean_test_loss_list = list()
test_acc_list = list()

# Initialize the batch size, the number of epochs, and the learning rate
batch_size = 500
epochs = 50
lr = 0.0002

# Initialize the weights
weights = init_weights(layers)

# Epoch for loop
for epoch in range(epochs):
    
    # Initialize the layers
    h,o = init_layer(layers,batch_size)
   
    # Initialize the training loss and accuracy for each epoch
    train_loss = 0
    train_acc = 0

    # Create a random permutation for shuffling
    shuffle = np.random.permutation(n_samples)
    
    # Shuffle dataset and create mini-batches for each epoch
    X_batches = np.array_split(X[shuffle],n_samples/batch_size)
    Y_batches = np.array_split(Y[shuffle],n_samples/batch_size)
    
    # Mini-batch for loop
    for b in range(int(n_samples/batch_size)):
        
        # Do feedforward step for mini-batch
        Y_hat_batch= feedforward(X_batches[b],weights,o,h)
        
        # Evaluate loss and accuracy for mini-batch
        train_loss += cross_entropy(Y_hat_batch,Y_batches[b])
        train_acc += accuracy(labeling(Y_hat_batch),Y_batches[b])       
    
        # Do backpropagation step for mini-batch
        dc_dw = backpropagation(Y_batches[b],Y_hat_batch,weights,h,o)

        # Update weights
        weights[1] = weights[1]-lr*dc_dw[1]
        weights[0] = weights[0]-lr*dc_dw[0]
        
    # Evalute loss and accuracy for the training dataset per epoch
    mean_train_loss = train_loss/len(Y_train)
    train_acc = (train_acc/len(X_batches))
    mean_train_loss_list.append(mean_train_loss)
    train_acc_list.append(train_acc)

    # Predictions for test dataset
    h_test,o_test = init_layer(layers,n_test)
    Y_hat_test= feedforward(X_test,weights,o_test,h_test)

    # Evaluate loss and accuracy for the test dataset per epoch
    mean_test_loss = cross_entropy(Y_hat_test,Y_test)/len(Y_test)
    test_acc = accuracy(labeling(Y_hat_test), Y_test)
    mean_test_loss_list.append(mean_test_loss)
    test_acc_list.append(test_acc)

    # Print statistics per epoch
    print(f"Epoch {epoch+1}: train_loss = {mean_train_loss:.3f} | train_acc = {train_acc:.3f} | test_loss = {mean_test_loss:.3f} | test_acc = {test_acc:.3f}" )


print(f"\nYour performance after {epochs} epoch(s) is: train_loss = {mean_train_loss:.3f} | train_acc = {train_acc:.3f} | test_loss = {mean_test_loss:.3f} | test_acc = {test_acc:.3f}\n")
print('Correct performance after 1 epoch is: train_loss = 2.314 | train_acc = 0.150 | test_loss = 2.248 | test_acc = 0.179')
print('Correct performance after 10 epochs is: train_loss = 1.746 | train_acc = 0.667 | test_loss = 1.731 | test_acc = 0.686')
print('Correct performance after 50 epochs is: train_loss = 1.592 | train_acc = 0.853 | test_loss = 1.592 | test_acc = 0.861')

##  损失和准确度曲线

一般来说，超参数如历时数、批量大小、学习率、隐藏层的数量和每层的神经元数量都需要针对每个应用进行仔细调整。在这里，我们展示了在一组给定的超参数下，损失和准确度函数作为历时数的函数是如何变化的，但也可以探索和绘制其他超参数值。


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))

ax[0].plot(mean_train_loss_list,label="Train loss")
ax[0].plot(mean_test_loss_list,label="Test loss")
ax[0].legend()
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")
ax[0].grid()

ax[1].plot(train_acc_list,label="Train accuracy")
ax[1].plot(test_acc_list,label="Test accuracy")
ax[1].legend()
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].grid()